In [1]:
import pyomo.environ as pyo

# Define sets
products = ["P1", "P2", "P3"]
subcontractable_products = ["P1", "P2"]

# Define parameters
smelting_time = {"P1": 6, "P2": 10, "P3": 8}  
mechanization_time = {"P1": 6, "P2": 3, "P3": 8} 
assembly_time = {"P1": 3, "P2": 2, "P3": 2}  

# Cost parameters
smelting_cost = {"P1": 0.30, "P2": 0.50, "P3": 0.40}
subcontracted_smelting_cost = {"P1": 0.50, "P2": 0.60}
mechanization_cost = {"P1": 0.20, "P2": 0.10, "P3": 0.27}
assembly_cost = {"P1": 0.30, "P2": 0.20, "P3": 0.20}

# Selling price
selling_price = {"P1": 1.50, "P2": 1.80, "P3": 1.97}

# Available production time
available_smelting_time = 8000
available_mechanization_time = 12000
available_assembly_time = 10000

# Create model
model = pyo.ConcreteModel()

# Define decision variables
model.x = pyo.Var(products, domain=pyo.NonNegativeReals) # In-house production
model.y = pyo.Var(subcontractable_products, domain=pyo.NonNegativeReals) # Subcontracted production

# Define objective function
model.profit = pyo.Objective(expr = 
    sum((selling_price[p] - smelting_cost[p] - mechanization_cost[p] - assembly_cost[p]) * model.x[p] for p in products) +
    sum((selling_price[p] - subcontracted_smelting_cost[p] - mechanization_cost[p] - assembly_cost[p]) * model.y[p] for p in subcontractable_products),
    sense=pyo.maximize)

# Define constraints
model.smelting_capacity = pyo.Constraint(expr = sum(smelting_time[p] * model.x[p] for p in products) <= available_smelting_time)

model.mechanization_capacity = pyo.Constraint(expr = 
    sum(mechanization_time[p] * (model.x[p] + model.y[p]) for p in subcontractable_products) + 
    sum(mechanization_time[p] * model.x[p] for p in products if p not in subcontractable_products) 
    <= available_mechanization_time)

model.assembly_capacity = pyo.Constraint(expr = 
    sum(assembly_time[p] * (model.x[p] + model.y[p]) for p in subcontractable_products) +
    sum(assembly_time[p] * model.x[p] for p in products if p not in subcontractable_products) 
    <= available_assembly_time)

# Solve the model
solver = pyo.SolverFactory('glpk')
solver.solve(model)

# Print results
print("Optimal Production Plan:")
for p in products:
    print(f"Product {p}: {model.x[p].value:.2f} (in-house)")
for p in subcontractable_products:
    print(f"Product {p}: {model.y[p].value:.2f} (subcontracted)")

print(f"\nTotal Profit: ${model.profit():.2f}")

Optimal Production Plan:
Product P1: 0.00 (in-house)
Product P2: 800.00 (in-house)
Product P3: 0.00 (in-house)
Product P1: 0.00 (subcontracted)
Product P2: 3200.00 (subcontracted)

Total Profit: $3680.00
